# Get the AST structure from file generated by CodeSensor-Complete

### Actual translation from 2 files (True & False - Training set)

In [ ]:
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec

In [ ]:
### TRUE

import ast
k = open('myCS/output/train_true.txt','r')
length = len(k.readlines())
k = open('myCS/output/train_true.txt','r')
all1 = []


In [ ]:
### TRUE

for i in range(0,length):
    try:
        all1.append(ast.literal_eval(k.readline()[:-1]))
        print('\r',i,end='')
    except:
        next

In [ ]:
### TRUE

true_df = pd.DataFrame(list(range(0,len(all1))), columns=['id'])
true_df['code'] = all1
true_df['label'] = np.repeat(0,len(all1)).tolist()

In [3]:
### FALSE

import ast
k = open('myCS/output/train_false.txt','r')
length = len(k.readlines())
k = open('myCS/output/train_false.txt','r')
all2 = []


In [ ]:
### FALSE

for i in range(0,length):
    try:
        all2.append(ast.literal_eval(k.readline()[:-1]))
        print('\r',i,end='')
    except:
        next

In [5]:
### FALSE

false_df = pd.DataFrame(list(range(0,len(all2))), columns=['id'])
false_df['code'] = all2
false_df['label'] = np.repeat(0,len(all2)).tolist()

### Combine TRUE & FALSE and save

In [6]:
## Save

final_df = true_df.append(false_df,ignore_index=True)
final_df.to_pickle('output/train.pickle')

### Train the Word2vec

In [1]:
## Load train data

train = pd.read_pickle('output/train.pickle')
train = train.reset_index()

In [2]:
## Function to flat the nested list
def flatten(x):
    return(list(pd.core.common.flatten(x)))

In [3]:
corpus = train.code.apply(flatten)

In [24]:
## Make sure every token is in string type
for line in corpus:
    for j in range(0,len(line)):
        if type(line[j]) == int:
            line[j] = str(line[j])

In [26]:
## Train the Word2Vec model
w2v = Word2Vec(corpus, size=128, workers=16, sg=1, min_count=3)
w2v.save('output/node_w2v_128')

### Convert token to index using trained Word2Vec

In [1]:
w2v = Word2Vec.load('output/node_w2v_128')

def recurse(node):
    for i in range(0,len(node)):
        if type(node[i]) == str:
            node[i] = w2v.wv.vocab.get(node[i]).index if node[i] in w2v.wv.vocab else w2v.wv.vectors.shape[0]-1
        elif type(node[i]) == list:
            recurse(node[i])
        else:
            node[i] = w2v.wv.vocab.get(str(node[i])).index if str(node[i]) in w2v.wv.vocab else w2v.wv.vectors.shape[0]-1

### Save the final dataset

In [9]:
mydf = pd.read_pickle('output/train.pickle')
mydf.code.apply(recurse)
mydf = mydf.sample(frac=1, random_state=666)
mydf.to_pickle('output/train_final.pickle')